<a href="https://colab.research.google.com/github/chlin1/TimeSeries/blob/master/inception_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from numpy import array
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers import LeakyReLU

from keras.models import Model
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten

# Data

In [78]:
urlx = 'https://raw.githubusercontent.com/chlin1/TimeSeries/master/data/data_X.csv'
urly = 'https://raw.githubusercontent.com/chlin1/TimeSeries/master/data/data_Y.csv'
dfx = pd.read_csv(urlx)
dfy = pd.read_csv(urly)

scaler = MinMaxScaler(feature_range=(0, 1))
dfx = scaler.fit_transform(dfx)


# input parameters
sample = 2000 # number of samples
slide = 1 # stride 
subtimesteps = 25 # for CNN+LSTM
num_subtimesteps = 4 # for CNN+LSTM
ratio = 0.8 # train samples/test samples



timesteps = num_subtimesteps * subtimesteps # timesteps  


# reshape from [samples, timesteps] into [samples, timesteps, features]
datax=[]
datay=[]

for i in range(1,sample):
  #datax.append(dfx.iloc[i*slide:i*slide + timesteps, 1:].values)
  datax.append(dfx[i*slide:i*slide + timesteps, 1:])
  datay.append(dfy.iloc[i*slide + timesteps - 1, 1:].values)
  
X = array(datax)
y = array(datay).flatten()

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
dummy_y = np_utils.to_categorical(encoded_Y)
X_train0, X_test0 = X[:int(sample*ratio)], X[int(sample*ratio):]
y_train, y_test = dummy_y[:int(sample*ratio)], dummy_y[int(sample*ratio):]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [79]:
print(X_train0.shape,X_test0.shape)
print(y_train.shape,y_test.shape)
X_train0.shape[1]

(1600, 100, 40) (399, 100, 40)
(1600, 3) (399, 3)


100

In [0]:
X_train=np.reshape(X_train0, (*X_train0.shape,1))
X_test=np.reshape(X_test0, (*X_test0.shape,1))


In [81]:
print(X_train1.shape,X_test1.shape)
print(y_train.shape,y_test.shape)

(800, 100, 40, 1) (199, 100, 40, 1)
(1600, 3) (399, 3)


# Model

In [0]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [0]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)


In [84]:
timesteps, num_feature = X_train.shape[1], X_train.shape[2]
input_layer = Input(shape=(timesteps, num_feature, 1))

x = Conv2D(filters =16, kernel_size = (4, num_feature))(input_layer)
x = LeakyReLU(alpha=.01)(x) 
x = Conv2D(filters =16, kernel_size = (1, 1))(x)
x = LeakyReLU(alpha=.01)(x)
x = Conv2D(filters =16, kernel_size = (4, 1))(x)
x = LeakyReLU(alpha=.01)(x)
x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32)
x = TimeDistributed(Flatten())(x)
x = LSTM(64, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(3, activation='softmax', name='output')(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(input_layer, x)
model.compile(loss=['categorical_crossentropy'], loss_weights=[1], optimizer='adam', metrics=['accuracy'])

In [86]:
# fit model
epochs = 100  
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=50)

Train on 1600 samples, validate on 399 samples
Epoch 1/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.8929 - acc: 0.6631 - val_loss: 1.2175 - val_acc: 0.3960
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.8243 - acc: 0.6844 - val_loss: 1.2382 - val_acc: 0.3960
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.7960 - acc: 0.6837 - val_loss: 1.1606 - val_acc: 0.3960
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.7776 - acc: 0.6888 - val_loss: 1.1298 - val_acc: 0.2581
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.7563 - acc: 0.6806 - val_loss: 1.1781 - val_acc: 0.2581
Epoch 6/100
1600/1600 [==============================] - 9s 5ms/step - loss: 0.7440 - acc: 0.6913 - val_loss: 1.2711 - val_acc: 0.2732
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.7366 - acc: 0.6994 - val_loss: 1.3262 - val_acc: 0.2732
Epoch 8

In [76]:
# fit model
epochs = 100  
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=50)
  

Train on 1600 samples, validate on 399 samples
Epoch 1/100
1600/1600 [==============================] - 10s 7ms/step - loss: 0.8561 - acc: 0.6669 - val_loss: 1.2422 - val_acc: 0.3960
Epoch 2/100
1600/1600 [==============================] - 9s 5ms/step - loss: 0.8136 - acc: 0.6850 - val_loss: 1.1851 - val_acc: 0.3960
Epoch 3/100
1600/1600 [==============================] - 9s 5ms/step - loss: 0.7948 - acc: 0.6850 - val_loss: 1.1439 - val_acc: 0.3960
Epoch 4/100
1600/1600 [==============================] - 9s 5ms/step - loss: 0.7669 - acc: 0.6850 - val_loss: 1.1599 - val_acc: 0.3960
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.9223 - acc: 0.6800 - val_loss: 1.6406 - val_acc: 0.3960
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.8266 - acc: 0.6850 - val_loss: 1.2537 - val_acc: 0.3960
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.8153 - acc: 0.6850 - val_loss: 1.2652 - val_acc: 0.3960
Epoch 8